In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

C:\Users\prit4\AppData\Local\Temp\ipykernel_18064\291067091.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

In [3]:
# Remove negative values from the dataset
for column in df.select_dtypes(include=['int64', 'float64']).columns:
    df = df[df[column] >= 0]

print('Negative values removed from the dataset.')

Negative values removed from the dataset.


In [4]:
# scaler = MinMaxScaler()
# scaled_features = scaler.fit_transform(df[['month', 'day', 'time', 'humidity', 'tempC']])
# print(scaled_features)

In [5]:
scaled_features = pd.DataFrame()
scaled_features['month'] = df['month'] / 12
scaled_features['day'] = df['day'] / 31
scaled_features['time'] = df['time'] / 24
scaled_features['humidity'] = df['humidity'] / 100
scaled_features['tempC'] = df['tempC'] / 50
print(scaled_features.head())

      month       day      time  humidity  tempC
0  0.083333  0.032258  0.000000      0.74   0.36
1  0.083333  0.032258  0.041667      0.78   0.36
2  0.083333  0.032258  0.083333      0.82   0.34
3  0.083333  0.032258  0.125000      0.85   0.34
4  0.083333  0.032258  0.166667      0.83   0.34


In [6]:
X = scaled_features
y = df['fan_speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  # Softmax activation for multi-class classification
])

In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)


Epoch 1/100
88/88 [==============================] - 1s 3ms/step - loss: 0.2353 - accuracy: 0.8957 - val_loss: 0.2370 - val_accuracy: 0.8898
Epoch 2/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2297 - accuracy: 0.8947 - val_loss: 0.2452 - val_accuracy: 0.8876
Epoch 3/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2293 - accuracy: 0.9022 - val_loss: 0.2419 - val_accuracy: 0.8905
Epoch 4/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2297 - accuracy: 0.8997 - val_loss: 0.2492 - val_accuracy: 0.8898
Epoch 5/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2284 - accuracy: 0.9007 - val_loss: 0.2378 - val_accuracy: 0.8905
Epoch 6/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2308 - accuracy: 0.9030 - val_loss: 0.2368 - val_accuracy: 0.8890
Epoch 7/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2288 - accuracy: 0.8990 - val_loss: 0.2499 - val_accuracy: 0.8855
Epoch 8/100
8

In [20]:
# Convert true classes to one-hot encoding
true_classes_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=6)
predicted_classes = model.predict(X_test)
# print(predicted_classes)
# Calculate accuracy
accuracy = np.mean(np.argmax(predicted_classes, axis=1) == np.argmax(true_classes_one_hot, axis=1))
print("Test Accuracy (Threshold):", accuracy)


55/55 [==============================] - 0s 932us/step
Test Accuracy (Threshold): 0.9049516220830962


In [21]:
model.save('model.h5')

c:\Users\prit4\OneDrive\Desktop\stuff\active_Github_repos\Mini_project\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
